# Fundamentos de ETL

1) EXTRAÇÃO

In [ ]:
import pandas as pd

# ler csv, diferenciar os dados que são do tipo date, garantir ordem y-m-d nas datas.
df = pd.read_csv('ocorrencia_2010_2020.csv', parse_dates = ['ocorrencia_dia'], dayfirst = True)
df

# retorno dos tipos de dado existentes em cada coluna
df.dtypes

# retorno apenas dos meses
df.ocorrencia_dia.dt.month

# primeiras dez linhas
df.head(10)

# df.tail(10) --> últimas dez linhas

2) VALIDAÇÃO DA EXTRAÇÃO


In [ ]:
pip install pandera 

# lib útil pra validação de dados
import pandera as pa


In [ ]:
df = pd.read_csv('ocorrencia_2010_2020.csv', parse_dates = ['ocorrencia_dia'], dayfirst = True)
df.head(10)

# esquema a ser validado --> identificação de erros
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String, pa.Check.str_length(2,2)),
        'ocorrencia_aerodromo': pa.Column(pa.String),
        'ocorrencia_dia': pa.Column(pa.DateTime),
        'ocorrencia_hora': pa.Column(pa.String, nullable = True)
    }
)

schema.validate(df)




3) LIMPEZA DE DADOS

3.1 Fazendo algumas alterações

In [ ]:
df.loc[0:3]                    # dados das três primeiras linhas
df.loc[[10,40]]                # dados das linhas 10 e 40
df.loc[:, 'ocorrencia_uf']     # todos os dados da coluna ocorrencia_uf

df.codigo_ocorrencia.is_unique     # observar se há repetição de texto/valor

# transformar a coluna em índice para a tabela como um todo
df.set_index('codigo_ocorrencia', inplace = True)

# utilizando codigo_ocorrencia para retornar as infos de uma linha
df.loc[40324] 

# voltar ao indice original
df.reset_index(drop = True, inplace = True)
df.head()

#### alterando dados

# deixando vazia a célula com asterisco da primeira linha de ocorrencia_aerodromo
df.loc[0,'ocorrencia_aerodromo'] = ''
df.head(1)

# alterando todos os dados de uma linha
df.loc[1] = 20  
df.head(2)

# alterando todos os dados de uma coluna
df.loc[:, 'ocorrencia_dia'] = 10  
df.head()

# criando coluna de backup de segurança para não perder dados originais!!!
df['ocorrencia_uf_bkp'] = df.ocorrencia_uf  
df

# alterando a classificação dos incidentes
# as ocorrencias de SP têm todas a classificação "GRAVE"
df.loc[df.ocorrencia_uf == 'SP', ['ocorrencia_classificacao']] = 'GRAVE'
df.loc[df.ocorrencia_uf == 'SP']

In [ ]:
# se esse código for rodado, retornará o dataset original, porque as alterações 
# feitas no bloco anterior dizem respeito apenas ao dataframe
df = pd.read_csv('ocorrencia_2010_2020.csv', parse_dates = ['ocorrencia_dia'], dayfirst = True)
df

3.2 Limpeza de dados

In [ ]:
# agora sim, depois de fazer as modificações desejadas, porém mudando de ideia
# e voltando ao dataset original

### limpando dados

''' Utilizando filtro de excel no csv analisado percebemos que os seguintes dados
precisam ser limpados do dataframe:

ocorrencia_uf
**

ocorrencia_aerodromo
****
*****
####
####!

ocorrencia_hora
NULL
'''
# opção 1 -- alterando de modo específico para <NA>
# df.loc[df.'ocorrencia_aerodromo' == '****', ['ocorrencia_aerodromo']] = pd.NA

# opção 2 -- alterando no geral
df.replace(['**', '****', '*****', '####', '####!', 'NULL'], pd.NA, inplace = True)

# saber quantos NA's há no dataframe
df.isna().sum()       # ou df.isnull().sum()  

# replace os NA's com alguma coisa
# atenção à necessidade de usar o "inplace = True"
df.fillna(0, inplace = True)

# replace NA's ou alguma coisa somente em algumas colunas
df.fillna(value = {'ocorrencia_classificacao':30}, inplace = True)

# exclusão de uma coluna de backup que não terá mais uso
# obs. utiliza-se "axis = 1" para especificar que se trata da coluna. Isso porque,
# por padrão, se nenhum parâmetro for escrito o python considera "axis = 0" que 
# se refere ao eixo horizontal, i.e., às linhas.
df['ocorrencia_hora_bkp'] = df.ocorrencia_hora
df.drop(['ocorrencia_hora_bkp'],  axis = 1, inplace = True)

# remove a LINHA TODA em que um valor nulo se encontrava
df.dropna()

# remove linhas duplicadas
df.drop_duplicates(inplace = True)

4) PARTE FINAL: TRANSFORMAÇÃO


In [ ]:
import pandas as pd
import pandera as pa

In [ ]:
# adicionando na leitura do dataframe (e antes da validação) o trabalho de 
# limpeza já feito. (Apenas porque nesse caso é possível fazer isso.)

valores_ausentes = ['**', '****', '*****', '####', '####!', 'NULL']
df = pd.read_csv('ocorrencia_2010_2020.csv', parse_dates = ['ocorrencia_dia'], dayfirst = True, na_values = valores_ausentes)

schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String, pa.Check.str_length(2,2), nullable = True),
        'ocorrencia_aerodromo': pa.Column(pa.String, nullable = True),
        'ocorrencia_dia': pa.Column(pa.DateTime),
        'ocorrencia_hora': pa.Column(pa.String, nullable = True)
    }
)

schema.validate(df)

In [ ]:
# utilização da variavel filtro para facilitar leitura
# e localização de infos
# ex. localizando células com valor nulo
filtro = df.ocorrencia_uf.is_null()
df.loc[filtro]

filtro = df.ocorrencia_aerodromo.is_null()
df.loc[filtro]

# count por default não conta células nulas
df.count()

In [ ]:
### aprimorando filtros

# ocorrências no rio de janeiro + nome das cidades
filtro = df.ocorrencia_uf == 'RJ'
df.loc[filtro, ['ocorrencia_uf', 'ocorrencia_cidade']]

# todas as infos sobre ocorrências graves
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro]

# ocorrências graves no estado de Minas Gerais
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'MG'
df.loc[filtro1 & filtro2]

# ocorrências graves OU no estado do Amazonas
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'AM'
df.loc[filtro1 | filtro2]

# ((ocorrências graves) ou (ocorrências comuns)) no estado de Goiás
filtro1 = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE')
filtro2 = df.ocorrencia_uf == 'GO'
df.loc[filtro1 & filtro2]

'''forma mais simples de fazer o código acima com .isin:

# ((ocorrências graves) ou (ocorrências comuns)) no estado de Goiás
filtro1 = (df.ocorrencia_classificacao.isin == (['INCIDENTE GRAVE', 'INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'GO'
df.loc[filtro1 & filtro2]
'''

# análise parcial
# ex. todas as ocorrências cuja cidade começa com C
filtro = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro, 'codigo_ocorrencia']

# ex. todas as ocorrências cuja cidade termina com 'MA'
filtro = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro, 'codigo_ocorrencia']

# ex. todas as ocorrências cuja uf CONTÉM 'S' (em qualquer posição)
filtro = df.ocorrencia_uf.str.contains('S')
df.loc[filtro]

# ex. todas as ocorrências cuja cidade CONTÉM 'S' ou 'PA' (em qualquer posição)
filtro = df.ocorrencia_uf.str.contains('S | PA')
df.loc[filtro]

# ocorrencias de 2015
# ocorrencia_dia é do tipo DateTime
filtro = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro] 

# ocorrencias de dezembro de 2016
filtro1 = df.ocorrencia_dia.dt.year == 2016
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

''' forma mais compacta do código acima:

# ocorrencias de dezembro de 2016
filtro1 = (df.ocorrencia_dia.dt.year == 2016) & (df.ocorrencia_dia.dt.month == 12)
df.loc[filtro]
'''
# ocorrencias de dezembro de 2016 com código iniciando em 5
filtro1 = df.ocorrencia_dia.dt.year == 2016
filtro2 = df.ocorrencia_dia.dt.month == 12
filtro3 = df.ocorrencia_codigo.str[0] == '5'
df.loc[filtro1 & filtro2 & filtro3]

# ocorrencias de julho de 2014 entre os dias 5 e 10
filtroAno = df.ocorrencia_dia.dt.year == 2014
filtroMes = df.ocorrencia_dia.dt.month == 7
filtroDia = (df.ocorrencia_dia.dt.day > 4) & (df.ocorrencia_dia.dt.day < 11)
df.loc[filtroAno & filtroMes & filtroDia]

# ocorrencias de julho de 2014 entre meia noite do dia 5 e 10h da manhã do dia 7

# unindo as colunas de data e horário 
# utilizando astype para não dar erro nos tipos de dado e utilizando to_datetime
# para que tenhas células com datetime, não com strings
df['ocorrencia_dia_hora'] = pd.to_datime(df.ocorrencia_dia.astype(str) + '' + df.ocorrencia_hora)

filtro1 = df.ocorrencia_dia_hora >= '2014-07-05 00:00:00'
filtro2 = df.ocorrencia_dia_hora <= '2014-07-07 10:00:00'
df.loc[filtro1 & filtro2]

In [ ]:
# agrupamento, como fazer

# atribuindo filtros a um novo dataframe --> informações sobre março de 2010
filtro1 = df.ocorrencia_dia.dt.year == 2010
filtro2 = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro1 & filtro2]
df201503

df201503.count()
# agrupando a partir de uma coluna e contando a partir de outra que não tenha dados nulos
df201503.groupby(['codigo_ocorrencia']).ocorrencia_hora.count()

# outra opção -- utilizar size() para agrupar e contar as linhas que foram agrupadas
df201503.groupby(['codigo_ocorrencia']).size()


df201503.groupby(['ocorrencia_classificacao']).size().sort_values()                    # ordem crescente
df201503.groupby(['ocorrencia_classificacao']).size().sort_values(ascending = False)   # ordem decrescente

######
# EX. agrupamento da região sudeste no ano de 2011
filtro1 = df.ocorrencia_dia.dt.year == 2011
filtro2 = df.ocorrencia_uf.isin(['SP', 'RJ', 'MG', 'ES'])
dfsudeste2011 = df.loc[filtro1 & filtro2] 
dfsudeste2011

# agrupando por tipo de ocorrência, descobrimos a quantidade de incidentes no 
# sudeste em 2011 e quantos de cada tipo
dfsudeste2011.groupyby(['ocorrencia_classificacao']).size()

# agrupando por tipo de ocorrência e depois por uf para saber a quantidade de cada tipo
# de incidente
dfsudeste2011.groupyby(['ocorrencia_classificacao', 'ocorrencia_uf']).size()

# agrupando por uf e depois por tipo de ocorrência para saber a quantidade de cada tipo
# de incidente
dfsudeste2011.groupyby(['ocorrencia_uf', 'ocorrencia_classificacao']).size()

# filtrando as ocorrência identificadas na cidade do Rio de Janeiro
filtro = dfsudeste2011.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2011.loc[filtro]

# agrupando o total de recomendações por mês e por cidade
filtro = dfsudeste.total_recomendacoes > 0
dfsudeste2011.loc[filtro].groupby(['ocorrencia_cidade', dfsudeste2011.ocorrencia_dia.dt.month]).total_recomendacoes.sum()